In [ ]:
from vfield import render_flow_field
import torch
import matplotlib.pyplot as plt
import lic
import numpy as np
from scipy.interpolate import griddata

In [ ]:
def plot_vector_field(grid, displacement):
    """Plot a vector field using matplotlib."""

    x = grid[:, 0].numpy()
    y = grid[:, 1].numpy()
    u = displacement[:, 0].numpy()
    v = displacement[:, 1].numpy()

    # color the vector field based on the magnitude of the displacement
    magnitude = np.sqrt(u**2 + v**2)
    plt.quiver(x, y, u, v, magnitude, cmap='viridis',
            angles='xy', scale_units='xy', scale=1, alpha=0.5)

    plt.xlim(np.min(x), np.max(x))
    plt.ylim(np.min(y), np.max(y))
    plt.axis('off')
    plt.gca().set_aspect('equal', adjustable='box')

def render_lic(grid, displacement, resolution=1024, length=30, normalize=True) -> np.ndarray:
    """
    Create a Line Integral Convolution (LIC) visualization of a vector field.

    Args:
        grid: tensor of shape (N, 2) representing the grid points
        displacement: tensor of shape (N, 2) representing the vector field at each grid point
        resolution: resolution of the output image (default: 1024)
        length: length of the LIC lines (default: 30)
        normalize: whether to normalize the vector field (default: True)
    
    Returns:
        lic_img: numpy array of shape (resolution, resolution) representing the LIC image
    """
    grid_np = grid.numpy()
    displacement_np = displacement.numpy()

    a = grid_np[:, 0].min()
    b = grid_np[:, 0].max()
    c = grid_np[:, 1].min()
    d = grid_np[:, 1].max()

    # create high-resolution grid
    x_lic, y_lic = np.mgrid[a:b:resolution*1j, c:d:resolution*1j]
    
    # analytic test field: a clockwise vortex centred at (0,0)
    # u = -y_lic
    # v = x_lic

    # create points for interpolation (flatten the high-res grid)
    points_lic = np.column_stack((x_lic.ravel(), y_lic.ravel()))
    
    # interpolate displacement components
    u_interp = griddata(grid_np, displacement_np[:, 0], points_lic, 
                        method='linear', fill_value=0.0)
    v_interp = griddata(grid_np, displacement_np[:, 1], points_lic, 
                        method='linear', fill_value=0.0)
    
    # reshape back to grid
    u = u_interp.reshape(resolution, resolution)
    v = v_interp.reshape(resolution, resolution)

    if normalize:
        mag = np.hypot(u, v)
        u = u / (mag + 1e-9)
        v = v / (mag + 1e-9)

    # render the LIC
    seed = lic.gen_seed((resolution, resolution), noise="white")
    lic_img = lic.lic(u, v, seed, length=length)

    lic_img = np.rot90(lic_img, k=3)  # rotate 90 degrees ccw
    lic_img = np.fliplr(lic_img)      # flip left-right
    
    return lic_img

In [ ]:
vfield = torch.load('../data/vectorfield.pt')

grid = vfield['grid']
displacement=vfield['displacement']

In [ ]:
# line integral convolution
lic_img = render_lic(grid, displacement, resolution=512, length=30, normalize=True)

# particle-based flow field
ff_img = render_flow_field(grid, displacement, particles=2000, 
                           bg_color=(255, 246, 251), trace_color=(25, 74, 103))

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(131)
plot_vector_field(grid, displacement)
plt.title('Matplotlib')

plt.subplot(132)
plt.imshow(lic_img, cmap='gray', origin='lower')
plt.title('Line Integral Convolution')
plt.axis('off')

plt.subplot(133)
plt.imshow(ff_img, origin='lower')
plt.title('Particle-based flow field', fontweight='bold')
plt.axis('off')

plt.tight_layout()
plt.show()